In [ ]:
from mastodon import Mastodon
from pyspark.sql import SparkSession
from kafka import KafkaProducer
import json
import time
import logging

# 1. Configuration du système de logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 2. Configuration de l'accès à l'API Mastodon
logging.info("Configuration de l'accès à l'API Mastodon...")
mastodon = Mastodon(
    access_token='m6d7yGLo1ScqQqfKehJz0YgzuIowJf2UsCK3fRvNBUY',  # Remplacez par votre jeton d'accès
    api_base_url='https://mastodon.social'  # L'URL de votre instance Mastodon
)

# 3. Créer une session Spark
logging.info("Création de la session Spark...")
spark = SparkSession.builder \
    .appName("MastodonToKafkaStream") \
    .getOrCreate()

# 4. Configuration du producteur Kafka
logging.info("Configuration du producteur Kafka...")
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# 5. Fonction pour récupérer les posts depuis l'API Mastodon
def fetch_mastodon_posts():
    logging.info("Récupération des posts depuis Mastodon...")
    # Récupérer les toots en temps réel (ici on récupère les 100 derniers toots)
    toots = mastodon.timeline_public(limit=100)

    for toot in toots:
        # Extraire les informations du toot
        toot_data = {
            'id': toot['id'],
            'username': toot['account']['username'],
            'display_name': toot['account']['display_name'],
            'content': toot['content'],
            'favourites_count': toot['favourites_count'],
            'reblogs_count': toot['reblogs_count'],
            'replies_count': toot['replies_count']
        }

        # Envoyer les données à Kafka
        producer.send('mastodon-toots', toot_data)
    logging.info(f"{len(toots)} toots envoyés à Kafka.")

# 6. Stream en continu depuis Mastodon vers Kafka
def stream_mastodon_to_kafka():
    logging.info("Démarrage du stream Mastodon vers Kafka...")
    while True:
        try:
            fetch_mastodon_posts()
            logging.info("Attente de 30 secondes avant la prochaine récupération...")
            time.sleep(30)  # Attendre 30 secondes avant de récupérer les nouveaux posts
        except Exception as e:
            logging.error(f"Erreur lors du stream des données : {e}")

# 7. Lancer le stream
stream_mastodon_to_kafka()

# 8. Fermer le producteur Kafka à la fin (nécessaire si vous arrêtez le programme)
producer.close()
logging.info("Producteur Kafka fermé.")
